In [193]:
%matplotlib inline

# Standard imports
import copy
import itertools

# Scientific computing imports
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import pandas
import seaborn; seaborn.set()
import string
import scipy.stats as sci


# Import widget methods
from IPython.html.widgets import *

#Student Class

In [167]:
class Student(object):
    
    def __init__(self, school_id, cohort_id, student_id, student_ability=600, 
                 ability_rank=0, score_rank=0, app_strategy="it", college_budget=1000, age=0):
        """
        Constructor for Student class.  
          * student ability drawn from a truncated normal distributon **between 0 and 1**
          * student score is ability times 2000 plus or minus 200
          * ranks are established from all students in that cohort across high schools
          * app strategy is either Income Typical or Achievement Typical
          
        Note that we must "link" the Person to their "parent" Model object.
        """
        # Set model link and ID
        self.school_id = school_id
        self.cohort_id = cohort_id
        self.student_id = student_id
        
        # Set student parameters 
        self.student_ability = student_ability
        self.student_score = student_score
        self.ability_rank = ability_rank 
        self.score_rank = score_rank
        self.app_strategy = app_strategy
        self.college_budget = college_budget
        self.age = age 
        
    def take_test(self):
        
        
        
    
        

#Model Class

* constructor: class constructor, which "initializes" or "creates" the model when we call Model(). This is in the __init__ method.


In [207]:
class Model(object):
    """
    Model Class, which encapsulates the entire behavior in each run of the model
    """
    def __init__(self, num_schools, num_colleges, num_cohorts=3, prob_out=.1, min_sticker_price=10000, 
                 max_sticker_price=50000, min_net_price=0, max_net_price=50000, 
                 min_college_budget=0, max_college_budget=100000, at_prob_l=.2, at_prob_m=.5, 
                 at_prob_h=.8, switch_prob=.6):
        
        #Set Model parameters 
        self.num_schools = num_schools
        self.num_colleges = num_colleges
        self.num_cohorts = num_cohorts
        self.prob_out= prob_out
        self.min_sticker_price = min_sticker_price
        self.max_sticker_price = max_sticker_price
        self.min_net_price = min_net_price
        self.max_net_price = max_net_price
        self.min_college_budget = min_college_budget
        self.max_college_budget = max_college_budget
        self.at_prob_l = at_prob_l
        self.at_prob_m = at_prob_m
        self.at_prob_h = at_prob_h
        self.switch_prob = switch_prob 
        
        #Set empty dictionary for storing graph
        self.student_graphs = {}
        
        # Set our state variables
        self.students = [] 
        self.t = 0
        
        
        # Call our setup methods to initialize high school space, students 
        self.setup_hs_space()
        self.setup_students()
        
        
    def setup_hs_space(self):
        """
        Set up small world networks within highschools
        """
        school_list = string.ascii_uppercase

        # Iterate
        for school_id in school_list:
            # Set the school ID
            num_cohorts = 3
            prob_out = 0.1

            # Setup empty school dictionary key
            self.student_graphs[school_id] = {}

            # Iterate through each cohort
            for cohort_id in range(num_cohorts):
                # Create graph randomly
                num_students = np.random.randint(50, 100)
                num_friends = 2 + np.random.binomial(4, 0.5)
                g = nx.newman_watts_strogatz_graph(num_students, num_friends, prob_out)

                # Setup empty cohort dictionary key 
                self.student_graphs[school_id][cohort_id] = {}

                # Create students
                for node_id in g.nodes():
                    # Set a student class as an attribute
                    g.node[node_id] = Student(school_id, cohort_id, node_id, 
                                              student_ability=sci.truncnorm.rvs(0, 1, size=1)
                                              ability_rank=0, score_rank=0, app_strategy=0, 
                                              college_budget=1000, age=0))

                    # Set school-cohort graph into dictionary
                    self.student_graphs[school_id][cohort_id][node_id] = g 
                    # Add Students to list of students
    
    def setup_students(self): 
        """
        Method to setup our people 
        """
        for school_id in self.student_graphs:
            for cohort_id in self.student_graphs[school_id]:
                for student_id in self.student_graphs[school_id][cohort_id]:
                    self.students.append(Student(school_id=school_id,
                                                 cohort_id=cohort_id, student_id=student_id, 
                                                 student_ability=sci.truncnorm.rvs(0, 1, size=1), 
                                                 ability_rank=0)

SyntaxError: invalid syntax (<ipython-input-207-262379e184dd>, line 69)

In [202]:
x = Model(20, 20)
x.student_graphs

{'A': {0: {0: <networkx.classes.graph.Graph at 0x108fa3290>,
   1: <networkx.classes.graph.Graph at 0x108fa3290>,
   2: <networkx.classes.graph.Graph at 0x108fa3290>,
   3: <networkx.classes.graph.Graph at 0x108fa3290>,
   4: <networkx.classes.graph.Graph at 0x108fa3290>,
   5: <networkx.classes.graph.Graph at 0x108fa3290>,
   6: <networkx.classes.graph.Graph at 0x108fa3290>,
   7: <networkx.classes.graph.Graph at 0x108fa3290>,
   8: <networkx.classes.graph.Graph at 0x108fa3290>,
   9: <networkx.classes.graph.Graph at 0x108fa3290>,
   10: <networkx.classes.graph.Graph at 0x108fa3290>,
   11: <networkx.classes.graph.Graph at 0x108fa3290>,
   12: <networkx.classes.graph.Graph at 0x108fa3290>,
   13: <networkx.classes.graph.Graph at 0x108fa3290>,
   14: <networkx.classes.graph.Graph at 0x108fa3290>,
   15: <networkx.classes.graph.Graph at 0x108fa3290>,
   16: <networkx.classes.graph.Graph at 0x108fa3290>,
   17: <networkx.classes.graph.Graph at 0x108fa3290>,
   18: <networkx.classes.graph